In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/covid19-dataset/Covid Data.csv


# Here we will see the impact of PCA on Logistic Regression models

In [2]:
import matplotlib.pyplot as plt
import  seaborn as sns

In [5]:
df=pd.read_csv('/kaggle/input/covid19-dataset/Covid Data.csv')
df.head()

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,DATE_DIED,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,...,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL,ICU
0,2,1,1,1,03/05/2020,97,1,65,2,2,...,2,2,1,2,2,2,2,2,3,97
1,2,1,2,1,03/06/2020,97,1,72,97,2,...,2,2,1,2,2,1,1,2,5,97
2,2,1,2,2,09/06/2020,1,2,55,97,1,...,2,2,2,2,2,2,2,2,3,2
3,2,1,1,1,12/06/2020,97,2,53,2,2,...,2,2,2,2,2,2,2,2,7,97
4,2,1,2,1,21/06/2020,97,2,68,97,1,...,2,2,1,2,2,2,2,2,3,97


In [7]:
df.absshape

(1048575, 21)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 21 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   USMER                 1048575 non-null  int64 
 1   MEDICAL_UNIT          1048575 non-null  int64 
 2   SEX                   1048575 non-null  int64 
 3   PATIENT_TYPE          1048575 non-null  int64 
 4   DATE_DIED             1048575 non-null  object
 5   INTUBED               1048575 non-null  int64 
 6   PNEUMONIA             1048575 non-null  int64 
 7   AGE                   1048575 non-null  int64 
 8   PREGNANT              1048575 non-null  int64 
 9   DIABETES              1048575 non-null  int64 
 10  COPD                  1048575 non-null  int64 
 11  ASTHMA                1048575 non-null  int64 
 12  INMSUPR               1048575 non-null  int64 
 13  HIPERTENSION          1048575 non-null  int64 
 14  OTHER_DISEASE         1048575 non-null  int64 
 15

In [10]:
df['DATE_DIED'] = df['DATE_DIED'].replace('9999-99-99', pd.NaT)

# Convert the dates to datetime objects
df['DATE_DIED'] = pd.to_datetime(df['DATE_DIED'])

# Check the result
print(df['DATE_DIED'])

0         2020-03-05
1         2020-03-06
2         2020-09-06
3         2020-12-06
4         2020-06-21
             ...    
1048570          NaT
1048571          NaT
1048572          NaT
1048573          NaT
1048574          NaT
Name: DATE_DIED, Length: 1048575, dtype: datetime64[ns]


In [12]:
df=df.drop(columns='DATE_DIED')

In [13]:
df.head()

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,COPD,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL,ICU
0,2,1,1,1,97,1,65,2,2,2,2,2,1,2,2,2,2,2,3,97
1,2,1,2,1,97,1,72,97,2,2,2,2,1,2,2,1,1,2,5,97
2,2,1,2,2,1,2,55,97,1,2,2,2,2,2,2,2,2,2,3,2
3,2,1,1,1,97,2,53,2,2,2,2,2,2,2,2,2,2,2,7,97
4,2,1,2,1,97,2,68,97,1,2,2,2,1,2,2,2,2,2,3,97


In [14]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

In [15]:
X = df.drop(columns='CLASIFFICATION_FINAL')
y = df['CLASIFFICATION_FINAL']

In [30]:
df['CLASIFFICATION_FINAL'].value_counts()

7    499250
3    381527
6    128133
5     26091
1      8601
4      3122
2      1851
Name: CLASIFFICATION_FINAL, dtype: int64

In [17]:
from sklearn.preprocessing import StandardScaler ,MinMaxScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

array([[ 0.76275324, -2.14342559, -0.99852004, ..., -0.05008022,
        -0.0402647 ,  0.47377997],
       [ 0.76275324, -2.14342559,  1.00148216, ..., -0.24480886,
        -0.0402647 ,  0.47377997],
       [ 0.76275324, -2.14342559,  1.00148216, ..., -0.05008022,
        -0.0402647 , -2.10612542],
       ...,
       [ 0.76275324,  1.07954257,  1.00148216, ..., -0.05008022,
        -0.0402647 ,  0.47377997],
       [ 0.76275324,  1.07954257,  1.00148216, ..., -0.05008022,
        -0.0402647 ,  0.47377997],
       [ 0.76275324,  1.07954257,  1.00148216, ..., -0.05008022,
        -0.0402647 ,  0.47377997]])

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=30)

# The model is performing very low in accuracy with all features

In [20]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)
model.score(X_test, y_test)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.5367761008988389

# Applying PCA

In [21]:
pca = PCA(0.95)
X_pca = pca.fit_transform(X)
X_pca.shape

# Only 4 features are selected by PCA model

(1048575, 4)

In [22]:
pca.explained_variance_ratio_

array([0.49624473, 0.38095231, 0.0447092 , 0.04169951])

In [27]:
pca.explained_variance_

array([2831.88783108, 2173.95599962,  255.13911981,  237.96389011])

In [28]:
X_train_pca1, X_test_pca1, y_train2, y_test2 = train_test_split(X_pca, y, test_size=0.2, random_state=30)

# After applying PCA, model is still performing way low but you can see accuracy on both models have very less difference ,LR-53%, LR-PCA : 51%

In [35]:
model1 = LogisticRegression(max_iter=100)
model1.fit(X_train_pca1, y_train2)
model1.score(X_test_pca1, y_test2)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.5185322938273371

In [32]:
import xgboost as xgb

In [34]:
y_train2 = y_train2 - 1
y_test2 = y_test2 - 1

# Define XGBoost classifier and set hyperparameters
xgb_clf = xgb.XGBClassifier(
    max_depth=5,
    n_estimators=100,
    learning_rate=0.1,
    objective='multi:softmax',  # set objective to multi-class classification
    num_class=7,  # number of classes in target variable
    random_state=42
)

# Fit the classifier to the training data
xgb_clf.fit(X_train_pca1, y_train2)

# Make predictions on the testing data
y_pred = xgb_clf.predict(X_test_pca1)

# Evaluate the accuracy of the classifier
accuracy = accuracy_score(y_test2, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5322938273371004
